# Baseline

<img src="./baseline.png" alt="nn" width="1000" height="800">

# Our Model

<img src="./ourEmbeddingModel.png" alt="nn" width="1000" height="800">

## Baseline
    - Task: video-language alignment (Video-to-Text, Text-to-Video)
    - Goal: 
        - Adapting image-text pre-trained models to video-text pre-training (i.e. post-training) 
        - Video Proxy mechanism on the basis of CLIP (CLIP-ViP) 
    - Approach 
        - In-domain auxiliary data generation: 
            - to bridge language domain gaps between images and videos datasets 
            - introduce auxiliary captions into large-scale video-subtitle data to reduce the language domain gap between pre-training and downstream data 
                - pre-training: Image-Text learning 
                - downstream data: Video-Text learning 

        - Video Proxy Mechanism:
            - to enable the Vision Transformer (ViT) model for both image and video encoding 
            - Before feeding into CLIP, we concatenate path tokens with a set of learnable parameters called video proxy tokens
            - The output of the first video proxy token will be regarded as the video's representation 
    - Loss function:
        - info-NCE loss 

## Our Model with emotion embedding 
    - 
        

# Model

In [14]:
import torch.nn as nn
import torch
import numpy as np

In [7]:
data = [{
"video": "video7112.mp4",
"video_id": "video7112",
"caption": "while other friends too try and hitting the basket another is eager to achieve his fourth successful basket in basketball",
"duration": 18.35,
"emotion": 1,
"positive": 4.0,
"negative": 0.0,
"neutral": 0.0,
"joy": 4.0,
"trust": 3.0,
"surprise": 1.0,
"anticipation": 3.0,
"fear": 0.0,
"sadness": 0.0,
"disgust": 0.0,
"anger": 0.0
}]

In [5]:
embed_dim = 512
emotion_embedding = nn.Embedding(8, embed_dim)

In [6]:
emotion_embedding

Embedding(8, 512)

In [34]:
# Change non-zero values to 1, effectively binarizing the input
emotions = torch.tensor(np.array([4.0, 3.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0])).unsqueeze(0) 
print(emotions.shape) # (bs, 8)
if emotions is not None:
    emotions = torch.where(emotions > 0, torch.ones_like(emotions), torch.zeros_like(emotions))
    print(f"emotions: {emotions}\n") 

torch.Size([1, 8])
emotions: tensor([[1., 1., 1., 1., 0., 0., 0., 0.]], dtype=torch.float64)



In [35]:
# Retrieve all emotion embeddings
batch_size = 1 
all_emotion_embeds = emotion_embedding.weight.unsqueeze(0).repeat(batch_size, 1, 1)  # [batch_size, 8, embed_dim]

In [36]:
all_emotion_embeds.shape

torch.Size([1, 8, 512])

In [40]:
if emotions is not None:
    emotion_mask = emotions.unsqueeze(-1).type_as(all_emotion_embeds)  # [batch_size, 8, 1]
    print(f"emotion_mask: {emotion_mask}, shape = {emotion_mask.shape}\n")
    selected_emotion_embeds = all_emotion_embeds * emotion_mask  # [batch_size, 8, embed_dim]
    print(f"selected_motion_embeds: {selected_emotion_embeds.shape}\n")
    emotion_embeds = selected_emotion_embeds.sum(1) / (emotion_mask.sum(1) + 1e-8)  # [batch_size, embed_dim]

    print(f"emotion_embeds.shape: {emotion_embeds.shape}")

emotion_mask: tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.]]]), shape = torch.Size([1, 8, 1])

selected_motion_embeds: torch.Size([1, 8, 512])

emotion_embeds.shape: torch.Size([1, 512])


In [ ]:
seq_length = ? 

emotion_embeds = emotion_embeds.unsqueeze(1).expand(-1, seq_length, -1)  # [batch_size, seq_length, embed_dim]

position_embeddings = self.position_embedding(position_ids)

# embedding with additional emotion_embeds 
embeddings = inputs_embeds + position_embeddings + emotion_embeds

# Future Work 

1. Test Data Collection:
    - Image-captioning model OFA-Caption (Wang et al., 2022b) to generate one caption for the middle frame of each video in HD-VILA-100M
          - with max length of 16 words
          